<a href="https://colab.research.google.com/github/jinsusong/study-NLP-BERT/blob/main/text_extraction_with_bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###BERT (from HuggingFace Transformers) for Text Extraction

- SQuAD(Stanford Question-Answering Dataset)를 사용
- SQuAD에서 입력은 질문과 컨텍스트에 대한 단락으로 구성
- 목표는 질문에 답하는 단락에서 텍스트의 범위를 찾는 것
- 실제 답변 중 하나와 정확히 일치하는 예측의 백분율을 측정하는 "정확한 일치" 측정항목을 사용하여 이 데이터에 대한 성능을 평가

- BERT 모델을 미세 조정

    1. 컨텍스트와 질문을 BERT에 대한 입력으로 제공
    2. BERT의 은닉 상태와 같은 차원을 가진 두 벡터 S와 T를 취함.
    3. 각 토큰이 응답 범위의 시작과 끝이 될 확률을 계산함.
    4. 토큰이 답의 시작이 될 확률은 S와 BERT의 마지막 계층에 있는 토큰 표현 사이의 내적에 의해 주어지며 모든 토큰에 대한 소프트맥스가 뒤따름
    5. 토큰이 답의 끝이 될 확률은 벡터 T와 유사하게 계산됨

In [ ]:
!pip install tokenizers
!pip install transformers

In [ ]:
import os
import re
import json
import string
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tokenizers import BertWordPieceTokenizer
from transformers import BertTokenizer, TFBertModel, BertConfig

max_len = 384
configuration = BertConfig()  # default parameters and configuration for BERT